<h1>CNN-H1 4-22-2016</h1>

<strong>Abstract</strong>
Implementing the CNN-H1 using NN2 described in the paper: http://arxiv.org/pdf/1509.00244v1.pdf. 

<strong>Implementation</strong>
<ul>
<li>Normalizes images to 230x230 centered around the facial features and then takes the middle 165X120 portion</li>
<li>Validation split of 15%</li>
<li>fc6 layer has a length of 512</li>
<li>Noise with width 15</li>
<li>Number of people is 600</li>
</ul>

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import dlib
import os
import fnmatch
import math
import numpy as np
import matplotlib.pyplot as plt
import sys

from matplotlib.pyplot import imshow
from scipy.ndimage.filters import gaussian_filter
from scipy.spatial import distance
from scipy.misc import imrotate

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)


In [3]:
data_path = '../data/lfw_cropped'

input_shape = 165, 120
noise_width = 15
num_people = 600

<h2>Loading Files</h2>

In [4]:
def get_face_to_file_path_dict():
    face_to_file_paths_dict = {}
    
    for root, dirnames, filenames in os.walk(data_path):
        for dirname in dirnames:
            if dirname not in face_to_file_paths_dict:
                face_to_file_paths_dict[dirname] = []
            directory_path = os.path.join(data_path, dirname)
            for filename in os.listdir(directory_path):
                if filename.endswith('_image.npy'):
                    face_to_file_paths_dict[dirname].append(os.path.join(directory_path, filename))
                            
    return face_to_file_paths_dict

In [5]:
def get_face_to_file_paths_descending_list(face_to_file_paths_dict):
    return sorted(face_to_file_paths_dict.items(), key=lambda x: len(x[1]), reverse=True)

In [6]:
face_to_file_paths_dict = get_face_to_file_path_dict()

In [7]:
face_to_file_paths_list = get_face_to_file_paths_descending_list(face_to_file_paths_dict)[:num_people]

<h2>Helper Functions</h2>

In [8]:
def get_image_window(image, size, point):
    """
    Assume image is grey image
    """
    top = int(point[1] - size[0] / 2)
    left = int(point[0] - size[1] / 2)
    return image[top:top + size[0], left:left + size[1]]

In [9]:
def reflection(image):
    return np.array([list(reversed(row)) for row in image])

def partition(image, top_left, rows, cols):
    return np.array([row[top_left[1]:top_left[1] + cols] for row in image[top_left[0]:top_left[0] + rows]])

In [10]:
def get_random_noise_image(image, coords, width):
    """
    Apply random gaussian generated values
    and distribute them on gaussian distributed square
    centered on the coordinates passed in for the image
    """
    
    noise = np.zeros((image.shape[0], image.shape[1]))
    for coord in coords:
        # Convert coordinates to rows / columns
        apply_noise_at_point(coord[1], coord[0], noise, width)
    return np.clip(image + noise, 0, 1)

def apply_noise_at_point(x, y, noise, width):
    """
    Generate a block with a single random value placed at the center
    Apply the Gaussian filter with std of 4
    Place it on the noise array at the appropriate coordinates
    
    x represents the rows
    y represents the cols
    """
    
    block = np.zeros((width, width))
    block[width / 2, width / 2] = np.random.normal()
    block = gaussian_filter(block, sigma=4)
    
    x -= width / 2
    y -= width / 2
    
    x_end = min(noise.shape[0] - x, block.shape[0])
    x_start =  max(0, -x)

    y_end = min(noise.shape[1] - y, block.shape[1])
    y_start = max(0, -y)

    noise[max(0, x):x+block.shape[0], max(0, y):y+block.shape[1]] = block[x_start:x_end,y_start:y_end]

In [11]:
def apply_noise(image, coords):
    return get_random_noise_image(image, coords, noise_width)

In [12]:
def get_feature_points(f):
    f = f.replace('image', 'features')
    return np.load(f)

def get_landmark_points(f):   
    f = f.replace('image', 'landmarks')
    return np.load(f)

def image_read(f):
    return np.load(f)

<h2>Data Pre-Processing</h2>

In [13]:
images_by_class = [[(image_read(f), get_landmark_points(f), get_feature_points(f)) 
                    for f in x[1]] for x in face_to_file_paths_list]

In [14]:
# Create reflection with reflected coordinates
for i in range(len(images_by_class)):
    for j in range(len(images_by_class[i])):
        im, landmarks, features = images_by_class[i][j]
        new_features = [(im.shape[1] - p[0], p[1]) for p in features]
        new_landmarks = [(im.shape[1] - p[0], p[1]) for p in landmarks]
        new_landmarks[:3], new_landmarks[6:] = new_landmarks[6:], new_landmarks[:3]
        images_by_class[i].append((reflection(im), new_landmarks, new_features))

In [15]:
# Doubling the images and applying random gaussian noise
for i in range(len(images_by_class)):
    images_by_class[i] += images_by_class[i][:]
    images_by_class[i] = [(apply_noise(im, features), landmarks) 
                          for im, landmarks, features in images_by_class[i]]

In [16]:
# Get image of size 165x120, similar to the paper
for i in range(len(images_by_class)):
    images_by_class[i] = [get_image_window(im, input_shape, landmarks[4])
                          for im, landmarks in images_by_class[i]]

In [17]:
from collections import Counter
Counter([image.shape
            for images in images_by_class 
            for image in images if image.shape == input_shape
            ]).most_common(10)

[((165, 120), 26684)]

In [18]:
X_train = [image 
            for images in images_by_class 
            for image in images 
            if image.shape == input_shape
            ]
y_train = [images[0] 
            for images in enumerate(images_by_class) 
            for image in images[1] 
            if image.shape == input_shape
            ]

In [19]:
zipped = np.array(zip(X_train, y_train))
np.random.shuffle(zipped)

In [20]:
X_train = np.array([x[0] for x in zipped])
y_train = np.array([x[1] for x in zipped])

In [21]:
X_train = X_train.reshape(X_train.shape[0], 1, input_shape[0], input_shape[1])
Y_train = np_utils.to_categorical(y_train, len(images_by_class))

<h2> Training and Validation</h2>

In [22]:
def NN2(shape, nb_classes, nb_fc6):
    model = Sequential()
    
    # Layer 1
    model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=shape))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3))
    model.add(ZeroPadding2D((1,1)))
    model.add(AveragePooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(nb_fc6))
    model.add(Dense(nb_classes, activation='softmax'))
   
    return model

In [23]:
model_input_shape = (1, input_shape[0], input_shape[1])
nb_fc6 = 512

In [24]:
model = NN2(model_input_shape, num_people, nb_fc6)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

/usr/local/lib/python2.7/dist-packages/keras/models.py:392: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 22681 samples, validate on 4003 samples
Epoch 1/10
22681/22681 [==============================] - 739s - loss: 5.9070 - acc: 0.0703 - val_loss: 5.7301 - val_acc: 0.0862
Epoch 2/10
22681/22681 [==============================] - 739s - loss: 5.7608 - acc: 0.0780 - val_loss: 5.6625 - val_acc: 0.0862
Epoch 3/10
22681/22681 [==============================] - 739s - loss: 5.6310 - acc: 0.0808 - val_loss: 5.3689 - val_acc: 0.0989
Epoch 4/10
22681/22681 [==============================] - 739s - loss: 5.1812 - acc: 0.1073 - val_loss: 4.7195 - val_acc: 0.1566
Epoch 5/10
22681/22681 [==============================] - 740s - loss: 4.4595 - acc: 0.1895 - val_loss: 3.7226 - val_acc: 0.2928
Epoch 6/10
22681/22681 [==============================] - 741s - loss: 3.3033 - acc: 0.3371 - val_loss: 2.5481 - val_acc: 0.4667
Epoch 7/10
22681/22681 [==============================] - 741s - loss: 2.0065 - acc: 0.5407 - val_loss: 1.5800 - val_acc: 0.6273
Epoch 8/10
22681/22681 [========================

In [ ]:
json_string = model.to_json()
open('models/CNN-H1_epoch-10.json', 'w').write(json_string)
model.save_weights('models/CNN-H1_epoch-10.h5', overwrite=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001), metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 22681 samples, validate on 4003 samples
Epoch 1/10
22681/22681 [==============================] - 739s - loss: 0.0682 - acc: 0.9801 - val_loss: 0.2050 - val_acc: 0.9598
Epoch 2/10
  384/22681 [..............................] - ETA: 693s - loss: 0.0294 - acc: 0.9922

In [ ]:
json_string = model.to_json()
open('models/CNN-H1_epoch-20.json', 'w').write(json_string)
model.save_weights('models/CNN-H1_epoch-20.h5', overwrite=True)